# Дз 3. Тестирование статистических гипотез

<div class="alert alert-info" role="alert">

Важно!

* Плагиат карается обнулением работы
* При использовании чужого кода из открытых источников необходимо указать ссылку на источник
* В каждой задаче необходимо обосновывать выбор статистического критерия, формулировать нулевую и альтернативную гипотезу и делать интерпретацию получившихся результатов, а также содержательный вывод.
* Весь текст оформляется строго в ячейках типа markdown

## Задача 1. Осьминог Пауль (2 балла)


В период с 2008 по 2010 год в Германии жил Осьминог Пауль, который был известен тем, что мог предсказывать результаты матчей с участием сборной Германии.     
   
В таблице ниже вы можете видеть результаты его предсказаний: 


<a href="https://ibb.co/VWnc5yG"><img src="https://i.ibb.co/QFS3tBV/image.jpg" alt="image" border="0"></a>

Если считать, что случайным образом мы угадывали ли бы 50% матчей, то можно ли утверждать, что осьминог Пауль был талантливым экстрасенсом со способностями к предсказаниям?

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as sps
import statsmodels.stats.power as sts

In [4]:
p0 = 0.5
vec = [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]

In [5]:
std_Paul = np.std(vec)
mean_Paul = np.mean(vec)

H0: p_Paul > p0; H1: p_Paul < p0

Нулевая гипотеза: Пауль - талантливый экстрасенс; альтернативная: нет

Возьмем уровень значимости равным 0.05

In [13]:
z = (mean_Paul - p0) / (std_Paul / np.sqrt(len(vec)))
print("z-расчетная:", z)

z-расчетная: 3.4591775165361334


In [25]:
print("z-тест:", sps.norm.ppf([0.95], 0, 1))

z-тест: [1.64485363]


In [15]:
def t_test(crit, df):
    print("t-тест:",  sps.t.ppf([crit], df))

In [16]:
def z_test(crit):
    print("z-тест:", sps.norm.ppf([crit], 0, 1))

In [27]:
def t_des(mean1, mean2, std1, std2, n, m):
    std_sum = np.sqrt((std1 ** 2 * (n - 1) + std2 ** 2 * (m - 1)) / (n + m - 2))
    print("t-расчетная:", (mean1 - mean2) / (std_sum * np.sqrt(1 / n + 1 / m)))

In [32]:
def z_des(mean1, mean2, std1, std2, m, n):
    print("z-расчетная:", (mean1 - mean2) / np.sqrt(std1 ** 2 / n + std2 ** 2 / m))

Так как основная гипотеза p_Paul > p0, следовательно будем принимать ее, если z-расчетная > z-теста

Значит, Пауль-талантливый экстрасенс

## Задача 2.  Корм и котики (2 балла)

    
Новый корм для котиков рекламируется с помощью рекламного баннера. Отдел маркетинга разработал для рекламы новый баннер. Они считают, что он более красивый. Перед фирмой возникла необходимость проверить какой из двух баннеров лучше.

Для этого была создана специальная веб-форма. В этой веб-форме людям по очереди демонстрировали оба баннера и просили поставить лайк, если баннер нравится человеку. Если человеку нравятся оба баннера, он может поставить два лайка. Если ему не нравится ни один баннер, он может не ставить лайки вовсе. Собранные в конечном итоге данные находятся в табличке.

Можно ли утверждать, что оба баннера одинаково хороши? Сформулируйте статистическую гипотезу для проверки того, что оба баннера выбирают одинаково часто. Проверьте ее и сделайте выводы.


In [33]:
cat = pd.read_csv("banner_click_stat.csv", sep="\t", header=None)
cat

,0,1
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
...,...,...
995,0,0
996,0,0
997,0,0
998,0,0


In [35]:
mean0 = np.mean(cat[0])
mean1 = np.mean(cat[1])
std0 = np.std(cat[0])
std1 = np.std(cat[1])
n = len(cat[0])
m = len(cat[1])

In [36]:
z_des(mean0, mean1, std0, std1, n, m)

z-расчетная: -1.7271126578424774


Нулевая гипотеза: p_0 == p_1, альтернативная: p_0 < p_1 or p_0 > p_1

In [38]:
z_test(0.95)

z-тест: [1.64485363]


Чтобы не отклонять нулевую гипотезу, надо -z-тест < z-расчетная < z-тест

Но z-расчетная < -z-тест. Следовательно мы попадаем в область альтернативной гипотезы, когда второй баннер выбирают чаще

## Задача 3. Немного АВ-тестов (3 балла)
## Игра Cookie Cat

В этом задании мы поработаем с данными Cookie Cats - популярной игры-головоломки, которую разработала компания Tactile Entertainment. В этой игре игрок должен соединять одинаковые элементы в ряд, чтобы очистить поле и выиграть уровень. 

<p style="align: center;"><img align=center src="https://i.ytimg.com/vi/iPxZIp0cbJE/maxresdefault.jpg"  width=500></p>
 
 
<p style="align: center;"><img align=center src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/cc_gates.png"  width=500></p>


По мере прохождения уровней перед игроками появляются ворота, открытия которых нужно ждать какое-то время или пропускать их, совершая покупки в приложении. С точки зрения создателей эти ворота повышают интерес к игре и стимулируют к вложению денег в игру.


Но у создателей появляется важный вопрос: **где лучше всего ставить ворота?**
Первоначально первые ворота были размещены на уровне 30. Однако есть гипотеза, что лучше переместить их на уровень 40. В этом задании вам предлагается проанализировать результаты AB-теста, в котором создатели переместили первые ворота в Cookie Cats с 30 уровня на уровень 40. 

**Данные получены от 90 189 игроков, которые установили игру во время проведения AB-теста. Переменные:**

* `userid` - уникальный номер, идентифицирующий каждого игрока.

* `version` - был ли игрок помещен в контрольную группу (gate_30 - ворота на уровне 30) или в тестовую группу (gate_40 - ворота на уровне 40).

* `sum_gamerounds` - количество игровых раундов, сыгранных игроком в течение первой недели после установки
* `retention_1` - вернулся ли игрок в игру через 1 день после установки
* `retention_7` - вернулся ли игрок в игру через 7 дней после установки

Когда игрок устанавливал игру, ему случайным образом назначали `gate_30` или `gate_40`.

In [52]:
import pandas as pd
df = pd.read_csv('cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [54]:
print("Oбьем выборки:", round(sts.TTestIndPower().solve_power(0.05, power=0.7, nobs1=None, ratio=1.0, alpha=0.05)))

Oбьем выборки: 4939


### Вопрос 1: есть ли зависимость между возвращаемостью игроков и расположением ворот?

#### 1.1 Проверьте гипотезу для возвращения через день

In [77]:
gate30 = df[df['version'] == 'gate_30'].sample(n=4939, random_state=22)
gate40 = df[df['version'] == 'gate_40'].sample(n=4939)

In [78]:
mean_30ret1 = np.mean(gate30["retention_1"])
mean_40ret1 = np.mean(gate40["retention_1"])
std_30ret1 = np.std(gate30["retention_1"])
std_40ret1 = np.std(gate40["retention_1"])
n = 4939
m = 4939

In [79]:
t_des(mean_30ret1, mean_40ret1, std_30ret1, std_40ret1, n, m)

t-расчетная: -0.6874706185929663


Нулевая гипотеза: p30 == p40; альтернативная: p30 > p40

In [80]:
t_test(0.975, n + m - 2)

t-тест: [1.96020422]


Чтобы принять нулевую гипотезу, нужно -t-тест < t-расчетная < t-тест

Следовательно среднее количество игроков, которые возвращаются в игру спустя один день в разных версиях игры, одинаково

#### 2.2 Проверьте гипотезу для возвращения через семь дней

In [81]:
mean_30ret7 = np.mean(gate30["retention_7"])
mean_40ret7 = np.mean(gate40["retention_7"])
std_30ret7 = np.std(gate30["retention_7"])
std_40ret7 = np.std(gate40["retention_7"])
n = 4939
m = 4939

In [82]:
t_des(mean_30ret7, mean_40ret7, std_30ret7, std_40ret7, n, m)

t-расчетная: -0.3378930703078716


In [83]:
t_test(0.975, n + m - 2)

t-тест: [1.96020422]


Будем проверять те же гипотезы, что и в пункте про возвращение через день

Чтобы не отвергать нулевую гипотезу t-расчетная должна принадлежать промежутку [-t-тест, t-тест]

Можем понять, что это так, следовательно среднее количество игроков, которые возвращаются в игру спустя неделю в разных версиях игры, одинаково

### Вопрос 2: Отличается ли среднее количество раундов в группах gate_40 и gate_30?

In [84]:
mean_30sum = np.mean(gate30["sum_gamerounds"])
mean_40sum = np.mean(gate40["sum_gamerounds"])
std_30sum = np.std(gate30["sum_gamerounds"])
std_40sum = np.std(gate40["sum_gamerounds"])
n = 4939
m = 4939

In [85]:
t_des(mean_30sum, mean_40sum, std_30sum, std_40sum, n, m)

t-расчетная: -0.4643966537386319


Нулевая гипотеза: среднее количество раундов в группах равны. Альтернативная: либо в gate_40, либо в gate_30 больше

In [86]:
t_test(0.975, n + m - 2)

t-тест: [1.96020422]


Так как основная гипотеза остается той же, следовательно проверяем по тому же принципу

Среднее количество раундов в разных версиях игры одинаковы 

### Вопрос 3: Отличается ли медианное количество раундов в группах gate_40 и gate_30?

In [88]:
median_30sum = np.median(gate30["sum_gamerounds"])
median_40sum = np.median(gate40["sum_gamerounds"])
print(median_30sum, median_40sum)

17.0 17.0


## Задача 4: Время покемонов (3 балла)

В данной задаче вам будет необходимо проанализировать различные характеристики покемонов и сравнить их между собой

<p style="align: center;"><img align=center src="https://proprikol.ru/wp-content/uploads/2020/11/kartinki-pokemonov-2.jpg"  width=500></p>

Покемоны - это маленькие существа, которые сражаются друг с другом на соревнованиях. Все покемоны имеют разные характеристики (сила атаки, защиты и т. д.) и относятся к одному или двум так называемым классам (вода, огонь и т. д.). Профессор Оук является изобретателем Pokedex, портативного устройства, которое хранит информацию обо всех существующих покемонах. Как его ведущий специалист по данным, Вы только что получили от него запрос с просьбой осуществить аналитику данных на всех устройствах Pokedex.

### Описание данных

Профессор Оук скопировал все содержимое память одного устройства Pokedex, в результате чего получился набор данных, с которым Вы будете работать в этой задаче. В этом файле каждая строка представляет характеристики одного покемона:

* pid: Numeric - ID покемона
* HP: Numeric - Очки здоровья
* Attack: Numeric - Сила обычной атаки
* Defense: Numeric - Сила обычной защиты
* Sp. Atk: Numeric - Сила специальной атаки
* Sp. Def: Numeric - Сила специальной защиты
* Speed: Numeric - Скорость движений
* Legendary: Boolean - «True», если покемон редкий
* Class 1: Categorical - Класс покемона
* Class 2: Categorical - Класс покемона

In [14]:
pokemon = pd.read_csv("pokemon.csv")
pokemon

,pid,Name,Class 1,Class 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False
...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,True
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,True
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,True
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,True


Уровень значимости равен 0.05

### Вопрос 1

Профессор Оук подозревает, что покемоны в классе `grass` имеют более сильную обычную атаку, чем у покемонов в классе `rock`. Проверьте, прав ли он, и убедите его в своем выводе статистически. 

In [17]:
grass = np.array(pokemon[pokemon["Class 1"] == "Grass"]["Attack"])
rock = np.array(pokemon[pokemon["Class 1"] == "Rock"]["Attack"])

In [18]:
mean_grass = np.mean(grass)
mean_rock = np.mean(rock)
std_grass = np.std(grass)
std_rock = np.std(rock)
n = len(grass)
m = len(rock)

In [20]:
t_des(mean_grass, mean_rock, std_grass, std_rock, n, m)

t-расчетная: -3.483968342159054


Нулевая гипотеза: m_grass > m_rock, альтернативная: m_grass < m_rock

In [24]:
t_test(0.95, n + m - 2)

t-тест: [1.64580773]


Чтобы принимать нулевую гипотезу t-расчетная > t-тест. Но это не так, следовательно смотрим на левосторонний тест

t-расчетная < -t-тест. Это область альтернативной гипотезы, значит профессор Оук не прав

### Вопрос 2

Также профессор Оук очень хочет проверить, правда ли нет значимой разницы между силой обычной атаки и силой обычной защиты у покемонов. Помогите ему и ответьте на вопрос, подтвердив свои выводы с помощью проверки статистических гипотез

In [22]:
mean_attack = np.mean(pokemon["Attack"])
mean_def = np.mean(pokemon["Defense"])
std_attack = np.std(pokemon["Attack"])
std_def = np.std(pokemon["Defense"])
n = len(pokemon["Attack"])
m = len(pokemon["Defense"])

In [23]:
t_des(mean_attack, mean_def, std_attack, std_def, n, m)

t-расчетная: 3.2437920780405096


Нулевая гипотеза m_attack == m_def, альтернативная: m_attack != m_def

In [26]:
t_test(0.975, n + m - 2)

t-тест: [1.96144962]


Чтобы принять нулевую гипотезу, -t-тест < t-расчетная < t-тест

Но t-расчетная > t-тест, следовательно мы отвергаем нулевую гипотезу и попадаем в область альтернативной гипотезы, в которой      m_attack > m_def

### Вопрос 3

И последний вопрос, который мучает профессора Оука: правда ли, что средняя скорость покемонов превышает 66.6 ? Сформулируйте подходящие гипотезы и помогите профессору Оуку, ответив на вопрос.

In [29]:
mean_speed = np.mean(pokemon["Speed"])
mean_0 = 66.6
std_speed = np.std(pokemon["Speed"])
std_0 = 0
n = len(pokemon["Speed"])
m = 1

In [30]:
t_des(mean_speed, mean_0, std_speed, std_0, n, m)

t-расчетная: 0.05772449984384741


Нулевая гипотеза: m_speed > 66.6, альтернативная: m_speed == 66.6 or m_speed < 66.6

In [31]:
t_test(0.95, n + m - 2)

t-тест: [1.64676295]


Чтобы принимать нулевую гипотезу, нужно t-расчетная > t-тест. Но это не так

-t-тест < t-расчетная < t-тест, то есть мы попали в область альтернативной гипотезы, когда средняя скорость покемонов равна 66.6